In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['i', 'band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
        
    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i']) 
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)

    if 'i' in ds.dims:                                                                                               
        ds = ds.sel(i=ds.dims['i'])
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    tropopauses = {'mls': 40, 'saw': 45, 'trp': 37}
    
    if PARAM['atmpro'] == PARAM['atmpro']:
        atmpro = PARAM['atmpro']
    else:
        atmpro = 'mls'
        
    ilevels = [0, tropopauses[atmpro], -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'h2o': 'atmpro', 'co2': 0.0008}
band = [11]
atmpro = mls
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o h2o band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 1e-24
  molecule = h2o
  ng_adju = [0]
  ng_refs = [5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(500, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66)]
  wgt = [(0.4, 0.4, 0.5, 0.55, 0.85)]
o co2 band9
  atmpro = mls
  band = 9
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [3, 3]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 3000
  vmin = 1900
  w_diffuse = [(1.66, 1.66, 1.75), (1.75, 1.6, 1.85)]
  wgt = [(0.7, 0.8, 0.7), (0.8, 0.7, 0.8)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1  0.180084                    1  0.428739
0.000750        2  0.015082                    2  0.044906
0.001052        3  0.013386                    3  0.005177
0.001476        4  0.008144                    4 -0.011779
0.002070        5  0.002243                    5 -0.021386
0.002904        6 -0.003644                    6 -0.021002
0.004074        7 -0.009613                    7 -0.021948
0.005714        8 -0.015745                    8 -0.019497
0.008015        9 -0.022025                    9 -0.020058
0.011243       10 -0.028382                   10 -0.020048
0.015771       11 -0.034804                   11 -0.022592
0.022122       12 -0.041503                   12 -0.028197
0.031031       13 -0.049330                   13 -0.037423
0.043528       14 -0.062630                   14 -0.055808
0.061057       15 -0.078516                   15 -0.075393
0.085645       16 -0.084251                   16 -0.075916
0.120136       17 -0.079141                   17 -0.057786
0.168516       18 -0.072539                   18 -0.041232
0.236378       19 -0.057887                   19 -0.028759
0.331549       20 -0.028320                   20 -0.012271
0.465100       21  0.028157                   21  0.028794
0.652400       22  0.157885                   22  0.149390
0.915100       23  0.364645                   23  0.349143
1.283650       24  0.364731                   24  0.378565
1.800600       25  0.204901                   25  0.259304
2.525700       26  0.110152                   26  0.152770
3.542800       27  0.058908                   27  0.066920
4.969550       28  0.029409                   28  0.015770
6.970850       29  0.013910                   29  0.006837
9.778100       30  0.005820                   30  0.011586
13.715850      31  0.000906                   31  0.008262
19.239350      32 -0.001690                   32  0.001351
26.987250      33 -0.001627                   33 -0.002251
37.855300      34 -0.000898                   34 -0.001692
53.100050      35 -0.001014                   35 -0.001123
73.887500      36 -0.001510                   36 -0.001555
97.662500      37 -0.001733                   37 -0.001788
121.437500     38 -0.001793                   38 -0.001881
145.212500     39 -0.002315                   39 -0.002510
168.987500     40 -0.003197                   40 -0.003514
192.762500     41 -0.003664                   41 -0.004033
216.537500     42 -0.003436                   42 -0.003884
240.312500     43 -0.003337                   43 -0.003865
264.087500     44 -0.003355                   44 -0.003997
287.862500     45 -0.003422                   45 -0.003996
311.637500     46 -0.003429                   46 -0.003855
335.412500     47 -0.003280                   47 -0.003505
359.187500     48 -0.002968                   48 -0.003256
382.962500     49 -0.002484                   49 -0.002963
406.737500     50 -0.001823                   50 -0.002555
430.512500     51 -0.001011                   51 -0.001917
454.287500     52 -0.000104                   52 -0.001143
478.062500     53  0.000905                   53  0.000131
501.837500     54  0.002035                   54  0.001496
525.612500     55  0.003315                   55  0.003104
549.387500     56  0.004781                   56  0.004895
573.162500     57  0.006489                   57  0.006832
596.937500     58  0.008714                   58  0.009120
620.712500     59  0.010994                   59  0.011200
644.487500     60  0.013257                   60  0.012854
668.262500     61  0.015539                   61  0.014123
692.037500     62  0.017865                   62  0.014724
715.812500     63  0.020289                   63  0.015980
739.587500     64  0.022880                   64  0.017626
763.362500     65  0.025732                   65  0.019753
78

# Fluxes by Layer

CRD                        CLIRAD                \
                       flug      fldg     fnetg      flug          fldg   
pressure    level                                                         
0.000000    1     -3.981589  0.000000 -3.981589 -3.914322  8.381481e-09   
0.000624    2     -3.981595  0.000019 -3.981576 -3.914329  3.870412e-05   
0.000876    3     -3.981598  0.000022 -3.981576 -3.914332  4.341501e-05   
0.001229    4     -3.981601  0.000026 -3.981575 -3.914337  4.807748e-05   
0.001723    5     -3.981606  0.000031 -3.981575 -3.914343  5.309651e-05   
0.002417    6     -3.981611  0.000037 -3.981575 -3.914350  5.890163e-05   
0.003391    7     -3.981619  0.000045 -3.981575 -3.914360  6.635503e-05   
0.004757    8     -3.981630  0.000054 -3.981577 -3.914373  7.610671e-05   
0.006672    9     -3.981645  0.000065 -3.981580 -3.914391  8.944094e-05   
0.009359    10    -3.981666  0.000078 -3.981587 -3.914416  1.074618e-04   
0.013128    11    -3.981694  0.000094 -3.981600 -3.914449  1.322461e-04   
0.018415    12    -3.981734  0.000113 -3.981621 -3.914497  1.657859e-04   
0.025830    13    -3.981792  0.000134 -3.981658 -3.914566  2.103639e-04   
0.036232    14    -3.981879  0.000160 -3.981719 -3.914670  2.683105e-04   
0.050823    15    -3.982018  0.000191 -3.981827 -3.914839  3.407768e-04   
0.071291    16    -3.982259  0.000241 -3.982017 -3.915126  4.444997e-04   
0.100000    17    -3.982666  0.000362 -3.982304 -3.915587  6.472705e-04   
0.140271    18    -3.983316  0.000635 -3.982681 -3.916280  1.064706e-03   
0.196760    19    -3.984336  0.001169 -3.983167 -3.917304  1.812759e-03   
0.275997    20    -3.985901  0.002191 -3.983710 -3.918793  3.032346e-03   
0.387100    21    -3.988211  0.004129 -3.984083 -3.920888  4.965575e-03   
0.543100    22    -3.991341  0.007779 -3.983563 -3.923571  8.180538e-03   
0.761700    23    -3.994054  0.014580 -3.979474 -3.925511  1.398941e-02   
1.068500    24    -3.991691  0.025469 -3.966222 -3.922594  2.376203e-02   
1.498800    25    -3.983262  0.035631 -3.947631 -3.914388  3.485439e-02   
2.102400    26    -3.973518  0.040537 -3.932980 -3.904866  4.387491e-02   
2.949000    27    -3.964159  0.042225 -3.921934 -3.895421  4.975257e-02   
4.136600    28    -3.955614  0.041968 -3.913646 -3.886389  5.013538e-02   
5.802500    29    -3.948114  0.040271 -3.907843 -3.878089  4.494733e-02   
8.139200    30    -3.941556  0.037563 -3.903993 -3.870500  3.925138e-02   
11.417000   31    -3.935746  0.034013 -3.901733 -3.863586  3.683675e-02   
16.014700   32    -3.930884  0.029644 -3.901240 -3.857643  3.539326e-02   
22.464000   33    -3.927580  0.025049 -3.902531 -3.853289  3.207119e-02   
31.510500   34    -3.925804  0.021529 -3.904274 -3.850531  2.690058e-02   
44.200100   35    -3.925041  0.019416 -3.905625 -3.848814  2.264072e-02   
62.000000   36    -3.925686  0.017923 -3.907763 -3.848657  2.011587e-02   
85.775000   37    -3.928881  0.016865 -3.912016 -3.851615  1.869316e-02   
109.550000  38    -3.933639  0.016744 -3.916895 -3.856648  1.868944e-02   
133.325000  39    -3.939332  0.017387 -3.921945 -3.862938  1.968071e-02   
157.100000  40    -3.946577  0.018114 -3.928464 -3.871118  2.078959e-02   
180.875000  41    -3.956892  0.019424 -3.937468 -3.882830  2.260509e-02   
204.650000  42    -3.970913  0.023125 -3.947788 -3.898592  2.700737e-02   
228.425000  43    -3.988059  0.030596 -3.957464 -3.917681  3.515585e-02   
252.200000  44    -4.008447  0.041585 -3.966862 -3.940033  4.662202e-02   
275.975000  45    -4.032738  0.056426 -3.976311 -3.966308  6.163883e-02   
299.750000  46    -4.061855  0.075906 -3.985949 -3.997306  8.138288e-02   
323.525000  47    -4.095479  0.099874 -3.995605 -4.032838  1.060572e-01   
347.300000  48    -4.133510  0.128666 -4.004843 -4.072653  1.360003e-01   
371.075000  49    -4.176163  0.162961 -4.013201 -4.117058  1.712332e-01   
394.850000  50    -4.223592  0.203395 -4.020196 -4.166171  2.119996e-01   
418.625000  51    -4.275863  0.250533 -4.025329 -

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')